In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import warnings
import networkx as nx
import pickle

from bs4 import BeautifulSoup

%matplotlib inline
warnings.simplefilter(action='ignore', category=UserWarning)

# 1) Data Acquisition

We want to acquire a sub-network of the Wikipedia hyperlink network. In such a graph, each node is a Wikipedia page and there is a link between node a and node b is there is a link to page b on page a. This is a directed network but we will make it undirected later on.

The process of the acquisition is the following : 
* Start from an arbitrary root node (prefer and ambiguous page in order to get as many different communities as possible).
* Explore the page to get the intra-wiki links and get first nodes.
* For each first node, explore the intra-wiki links to get the second nodes.

We chose to scrap directly the HTML code of wikipedia pages rather than using the standard Wikipedia API because this API cannot fetch disambiguation pages natively and those are some pages we are interrested in as the list a large variety of links (more on that later). 

Moreover, scraping the HTML code of pages is really easy and straigtforward when we are simply looking for intra wiki links.

On Wikipedia pages, all the links are contained in the following HTML structure of tags: 
```html
<li>
    <a href:''>
    </a>
</li>
```
This is what is used in the following function that finds the intra Wiki links on a given page.

In [ ]:
def find_inner_links(page_link):
    """
    :param page_link: url of the current page
    :return: list of the intra wiki urls contained in the current page
    """
    links = []
    # Get the HTML code of the page.
    soup = BeautifulSoup(requests.get('https://en.wikipedia.org'+page_link).text)
    # Look for all the <li> tags contained in the page.
    li_tags = soup.find_all('li')  
    for li_tag in li_tags:
        # Look for all the <li> tags contained the current <a> tag.
        a_tags = li_tag.find_all('a')
        for a_tag in a_tags:
            try:
                # Check if the current <a> tag is an intra wiki link.
                if '/wiki/' == a_tag['href'][0:6]:
                    links.append(a_tag['href'])
            except KeyError:
                # In this case the <a> tag is not a link.
                pass
    return links

We use as `root_node` the disambiguation page [Jaguar](https://en.wikipedia.org/wiki/Jaguar_(disambiguation) as it lists a really wide variety of themes (animals, cars, music, films, weapons...). The aim is to scrap pages from as many different communities as possible.

In [ ]:
root_node = '/wiki/Jaguar_(disambiguation)'

In [ ]:
neighbors = {}  # This dict stores for each page the list of its neighbors.

In [ ]:
first_nodes = set(find_inner_links(root_node))
neighbors[root_node] = first_nodes

In [ ]:
second_nodes = set()
for link in first_nodes:
    tmp = find_inner_links(link)
    neighbors[link] = tmp
    second_nodes = second_nodes.union(set(tmp))

Create the set of all nodes.

In [ ]:
nodes = first_nodes.union(second_nodes)
nodes.add(origin)

Look for connections between second nodes and the rest of the nodes.

In [ ]:
counter = 0
for link in second_nodes:
    counter += 1
    if counter % 100 == 0:
        print('{}/{}'.format(counter, len(second_nodes)))
    tmp = find_inner_links(link)
    neighbors[link] = list(set(tmp).intersection(nodes))

#### Creating pickle files

As the scraping of the network takes quite some time (especially getting the inner connections), we store the results in pickle files.

In [ ]:
def save_obj(obj, name ):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
save_obj(nodes, 'nodes')
save_obj(first_nodes, 'first_nodes')
save_obj(second_nodes, 'second_nodes')
save_obj(neighbors, 'neighbors')

In [ ]:
nodes = load_obj('nodes')
first_nodes = load_obj('first_nodes')
second_nodes = load_obj('second_nodes')
neighbors = load_obj('neighbors')

### Network creation

Let's convert the collected network into a networkx instance which is quite handy to manipulate.

Let's 

In [ ]:
nodes = list(nodes)
nodes[0]

In [ ]:
#directed :
#g = nx.DiGraph(neighbors)

#undirected :
g = nx.Graph(neighbors)

In [ ]:
adj = nx.adjacency_matrix(g)
adj

In [ ]:
plt.spy(adj)

In [ ]:
plt.spy(nx.laplacian_matrix(g), precision=1)

Check if it's symmetric :

In [ ]:
(adj != adj.T)

In [ ]:
edges = g.edges(nodes[0])
edges = list(edges)

In [ ]:
len(edges)

In [ ]:
real_edges = neighbors[nodes[0]]
real_edges

In [ ]:
len(real_edges)

# 2) Data Exploration

In this part of the notebook, we provide some indicators of the data in order to understand what we'll be working on.

TODO: properties of the collected network

* Adjacency matrix
* Degrees distribution
* Average degree
* Diameter of the collected network
* (Pruning the collected network if necessary ?)
* Visualization of the network

In [ ]:
number_of_edges = 0
for i in neighbors.keys():  # for each node
    number_of_edges += len(neighbors[i])  # count the number of neighbors of the node

In [ ]:
print('Total number of nodes : {}'.format(len(nodes)))
print('Number of first nodes : {}'.format(len(first_nodes)))
print('Number of second nodes : {}'.format(len(second_nodes)))
print('Number of edges : {}'.format(s))

#### Degrees distribution

In [ ]:
degrees = adj.sum(axis=1)

plt.hist(degrees, bins=50);

In [ ]:
degrees_truncated = np.array(degrees)

degrees_truncated = degrees_truncated[degrees_truncated < 700]

plt.hist(degrees_truncated, bins=20);

#### Visualization

In [ ]:
nx.draw(g, node_size=5)

## Laplacian

In [ ]:
from scipy import sparse

In [ ]:
laplacian = sparse.csr_matrix(nx.laplacian_matrix(g), dtype=float)

In [ ]:
laplacian

In [ ]:
plt.spy(laplacian)

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(laplacian, k=10, which='SM')

In [ ]:
eigenvalues

In [ ]:
eigenvectors

In [ ]:
np.linalg.matrix_rank(laplacian)

In [ ]:
dense = laplacian.todense()

In [ ]:
dense

In [ ]:
dense[:, 3]

In [ ]:
dense[:, 4]

In [ ]:
dense[:, 5]

In [ ]:
dense[:, 6]

In [ ]:
plt.plot(eigenvalues, '.-', markersize=15);